# Problem Representation

## Variables

$ \forall \ Residents\ \times \ Rotations\ \times \ Weeks \ \exists \ scheduled \ [0,1]$

Implemented elements are signified by checked boxes.

## Constraints

### Primary constraints

- [x] Each resident must be scheduled on a rotation
- [x] Rotations will only be scheduled if they can fit the prescribed number of weeks `min_contig_wks`
- [x] Rotations can't have more residents scheduled than their `resident capacity`
- [x] Rotations that require staff must be staffed

### Only pursue if primary problem feasible

- [ ] Each resident must be scheduled on a rotation ***or on vacation every week***
- [ ] All new interns start in clinic or HS, then HS or clinic respectively.
- [ ] Rotations that require intern or senior must be restricted by role

## Optimization Targets

- [ ] Maximize resident elective preferences
- [ ] Maximize fit of vacation preferences

## Uncertain implementation elements

### Role specification
Many roles are restricted by year or intern/senior.

### Longitudinal requirements
Residents must complete certain rotations, but have all 3 years to complete this.

I suspect part of this would be pre-loading each resident's completed rotation weeks

### Fungibility
Some rotations meet requirements (or have limits) that span distinct rotation names.

For example, a resident can only work 6 months in ICU settings *total* across all 3 years, regardless of expressed preferences. There may be many rotations which count as ICU, but each rotation has its own resident staffing requirement and may be preferred by residents.

Perhaps House Staff is a better example - Orange and Green both have required staffing, but a resident need only complete House Staff rotations.

This appears to require a mapping of rotation -> rotation category. Each category may have a requirement (minimum) and maximum assigned.

### interval scheduling in `or-tools`
May function better, but reportedly less efficient. This has not been a problem for pure feasibility - a near-real problem completes in 1.5s.

### Variable rotation availability
variable availability of rotations - some rotations offered only in certain intervals

## Barriers to successful completion

- uncertain how to take into account the next year

# Implementation

## Imports

In [ ]:
import itertools as it

import numpy as np
import pandas as pd
from opre_tools import negated_bounded_span, print_full
from ortools.sat.python import cp_model

MAX_WKS = 52

### read data from excel file

I acknowledge this is odd - it's just for ease of creating and editing input.

In [ ]:
prefs = pd.read_excel("input.xlsx",sheet_name="preferences",index_col="resident_name")
rotations = pd.read_excel("input.xlsx",sheet_name="rotations",index_col='rotation_name')
residents = pd.read_excel("input.xlsx",sheet_name="residents",index_col='resident_name')
weeks = pd.read_excel("input.xlsx",sheet_name="weeks",index_col='week_n')
categories = pd.read_excel("input.xlsx",sheet_name="categories",index_col='category')

In [ ]:
prefs.head()

In [ ]:
rotations.head()

In [ ]:
residents.head(30)

In [ ]:
weeks.head()

In [ ]:
# sig_categories = categories.query("(category_min_wks > 1) and (category_max_wks < @MAX_WKS)")
# sig_categories

In [ ]:
# rotations_s_t_categories = pd.merge(sig_categories,rotations,left_index=True,right_on="category" )
# rotations_s_t_categories

In [ ]:
rotations[(rotations.min_wks > 0) & (rotations.required_level == "Senior")]

In [ ]:
for r, w in it.product(rotations.index, weeks.index):
    print(r, rotations.resident_capacity[r])
    break

In [ ]:
residents = residents[residents.resident_year == 3] # cut down for feasibility

In [ ]:
model = cp_model.CpModel()

d = model.NewBoolVarSeries(
    "bool_at_",
    pd.Index((list(it.product(residents.index, rotations.index, weeks.index)))),
).sort_index()  # makes a dummy pd.Series with strings that are labels

# every resident's week x must have exactly 1 rotation (has not had vacations incorporated)
for n, w in it.product(residents.index, weeks.index):
    model.AddExactlyOne(d.loc[pd.IndexSlice[n, :, w]])

# # Every rotation can only go one place each week >> JUST 
# for r, w in it.product(rotations.index, weeks.index):
#     model.AddAtMostOne(d.loc[pd.IndexSlice[:, r, w]])

# apply max capacity to each rotation for each rotation:week index
for r, w in it.product(rotations.index, weeks.index):
    model.Add(sum(d.loc[pd.IndexSlice[:, r, w]]) <= rotations.resident_capacity[r]*4)

# for each rotation, set max weeks to use
for rot_name, rot_tail in rotations.iterrows():
    for res in residents.index:
        if rot_tail["max_wks"] <= MAX_WKS:  # may be
            model.Add(
                sum(d.loc[pd.IndexSlice[res, rot_name, :]]) <= rot_tail["max_wks"]
            )

# # ^^^ set senior requirements
# for rot_name, rot_tail in rotations[(rotations.min_wks > 0) & (rotations.required_level == "Senior")].iterrows():
#     for res in residents[residents.resident_year > 1].index:
#         model.Add(sum(d.loc[pd.IndexSlice[res, rot_name, :]]) >= rot_tail.min_wks)

# # ^^^ set intern requirements
# for rot_name, rot_tail in rotations[(rotations.min_wks > 0) & (rotations.required_level == "Intern")].iterrows():
#     for res in residents[residents.resident_year == 1].index:
#         model.Add(sum(d.loc[pd.IndexSlice[res, rot_name, :]]) >= rot_tail.min_wks)

# for any rotations which require contiguous weeks
rotations_with_contig_reqs = rotations[rotations.min_contig_wks > 1]
for contig_rot, contig_rot_tail in rotations_with_contig_reqs.iterrows():
    for res in residents.index:
        hard_min = contig_rot_tail.min_contig_wks
        works = d.loc[pd.IndexSlice[res,contig_rot,:]]
        for length in range(1,hard_min):
            for start in range(len(works) - length + 1):
                model.AddBoolOr(negated_bounded_span(works,start,length))

print(model.ModelStats())

In [ ]:
# # TODO for each rotation with a category - could do
# for rot_name, rot_tail in rotations_s_t_categories.iterrows():
#     print(f"{rot_name} -  {rot_tail.category}")
#     for 

In [ ]:
# model.Maximize(
#     sum(
#         [
#             d.loc[pd.IndexSlice[res, rot, wk]] * prefs.loc[res, rot]
#             for res in residents.index
#             for rot in rotations.index
#             for wk in weeks.index
#         ]
#     )
# )

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 4
solver.parameters.log_search_progress = True
solver.log_callback = print
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
    print("optimal")
    # # The status tells us if we were able to compute a solution.
    # for n, w in it.product(residents, weeks):
    #     print(
    #         "{} {}\n{}".format(
    #             n,
    #             w,
    #             solver.Values(d.loc[pd.IndexSlice[n, :, w]])[
    #                 solver.Values(d.loc[pd.IndexSlice[n, :, w]]) != 0
    #             ],
    #         )
    #     )
    print("=====Stats:======")
    print(solver.SolutionInfo())
    print(solver.ResponseStats())
elif status == cp_model.FEASIBLE:
    print("feasible")
    # # The status tells us if we were able to compute a solution.
    # for n, w in it.product(residents, weeks):
    #     print(
    #         "{} {}\n{}".format(
    #             n,
    #             w,
    #             solver.Values(d.loc[pd.IndexSlice[n, :, w]])[
    #                 solver.Values(d.loc[pd.IndexSlice[n, :, w]]) != 0
    #             ],
    #         )
    #     )
    print("=====Stats:======")
    print(solver.SolutionInfo())
    print(solver.ResponseStats())
else:
    print("no solution")

In [ ]:
print(solver.Values(d)[solver.Values(d) == 1].sort_index(level=(0, 2)))
# solver.Values(d)[solver.Values(d) == 1].sort_index(level=(0, 2)).to_csv(
#     "D:/Informatics\\opre\\final_tabular.csv"
# )

print(
    solver.Values(d)[solver.Values(d) == 1].sort_index(level=(0, 2)).unstack().to_markdown()
)
# solver.Values(d)[solver.Values(d) == 1].sort_index(
#     level=(0, 2)
# ).unstack().to_csv("D:/Informatics\\opre\\final_grid.csv")

print_full(
    solver.Values(d)[solver.Values(d) == 1]
    .sort_index(level=(0, 2))
    .unstack()
    .reset_index()
    .melt(id_vars=["level_0", "level_1"])
    .query("value == 1")
    .set_index(["level_0", "variable"])
    .sort_index()["level_1"]
    .unstack()
)

In [ ]:
print_full(solver.Values(d)[solver.Values(d) == 1].sort_index(level=(0, 2)).unstack())

In [ ]:
print(
    solver.Values(d)[solver.Values(d) == 1]
    .sort_index(level=(0, 2))
    .unstack()
    .reset_index()
    .melt(id_vars=["level_0", "level_1"])
    .query("value == 1")
    .set_index(["level_0", "variable"])
    .sort_index()["level_1"]
    .unstack()
)

In [ ]:
(
    solver.Values(d)[solver.Values(d) == 1]
    .sort_index(level=(0, 2))
    .unstack()
    .reset_index()
    .melt(id_vars=["level_0", "level_1"])
    .query("value == 1")
    .set_index(["level_0", "variable"])
    .sort_index()["level_1"]
    .unstack()
    .to_excel("output_schedule.xlsx")
)